# 列舉所有資產的riskiness r

In [2]:
import sympy
from pandas import Series,DataFrame
from scipy.optimize import fsolve
import math
import time
import pandas as pd
import numpy  as  np
import os

# 無風險利率
* 我們須扣除無風險利率，得到超額報酬率以計算Riskiness r
* 目前使用LIBOR [1個月利率](https://www.global-rates.com/interest-rates/libor/american-dollar/usd-libor-interest-rate-overnight.aspx)

In [3]:
rfrate = 2.45785 / 12 /100
rfrate

0.0020482083333333333

# 關於超額報酬的數量級:
* 將要帶入公式的數值調整到介在0-100間，目的是為了讓大部分的Riskiness r介在0-100間。

In [4]:
def f1(x,arr_returns):
    return sum( np.exp(-1*arr_returns/x) ) - len(arr_returns)

In [5]:
def get_riskiness_r(guess , arr_returns):
    while (guess<1000):
        risk2 = fsolve(f1,guess,arr_returns)  #引用函數f1
        if risk2[0] != guess:
            break
        guess = guess*10
    return risk2[0]

# 計算Riskiness R，並依照R值排序，傳回前十名的ETF

In [6]:
def get_top5_etf(data_file):
    # 讀檔
    #data_file = '../../HW1/ETF_data_set/return_data/Alternatives ETF List (35)/Alternatives ETF List (35)_return.csv';
    df = pd.read_csv(data_file)
    etf_list = list(df.columns)[2:] #<------------------------------加上日期修改
    df = df.iloc[:,2:]
    
    # 計算每支ETF的riskiness r
    etf_dic = {}
    for etf_name in etf_list:
        # 過濾平均值正負
        #if df[etf_name].mean(axis=0) < 0:
        #    continue
        #--------------------------------------------------------------------------------------
        
        #這邊要加入時間
        
        
        arr_returns = df[etf_name] - rfrate  
        #arr_returns = arr_returns*1000      #調整數量級
        
        
        
        
        
        #----------------------------------------------------------------------------------------
        guess = 10**(-5)
        #### 列出細項
        #print('%s指標值:  %f' %(etf_name,risk2[0]) )
        etf_dic[etf_name] = get_riskiness_r(guess,arr_returns)
    
    # ETF依照riskiness r排序
    sorted_x = sorted(etf_dic.items(), key=lambda kv: kv[1])
        #sorted_reversed_x = list(reversed(sorted_x))
    
    # 取前5名
    top5 = sorted_x[:5]
        # top10 = sorted_reversed_x[:10]
    #print(top10)
    
    # 取前50的ETF名稱
    etf_top5 = [etf[0] for etf in top5]
    
    #前10的平均
    top5_average = df[etf_top5].mean(axis=1)
    
    #print(etf_top10)
    return etf_top5 , top5_average

# 讀取全部ETF資料夾

In [9]:
data_path = 'C:\\Users\\USER\\github\\HW1--data-mining\\FinalProject\\2014to2018RE_data\\'
allgroup_top5_return = DataFrame()
allgroup_top5_dict = dict()
for subdir in os.listdir(data_path):
    subdir_path = os.path.join(data_path, subdir)
    if os.path.exists(subdir_path) and os.path.isdir(subdir_path):
        file = subdir + '_return.csv'
        file_path = os.path.join(subdir_path, file)
        #print(file_path)
        if os.path.isfile(file_path):
            # 提出[前10名的名稱,前10名的平均各期報酬]
            [ top5_etf,allgroup_top5_return[subdir] ] = get_top5_etf(file_path)
            print('Group:',subdir)
            print('Top10:',top5_etf)
            print('================')
            allgroup_top5_dict[subdir] = top5_etf
            df = pd.read_csv(file_path)
            etf_date = df['Date']

C:\Users\USER\Anaconda3\lib\site-packages\scipy\optimize\minpack.py:162: RuntimeWarning: The iteration is not making good progress, as measured by the 
  improvement from the last ten iterations.
  warnings.warn(msg, RuntimeWarning)
C:\Users\USER\Anaconda3\lib\site-packages\scipy\optimize\minpack.py:162: RuntimeWarning: The number of calls to function has reached maxfev = 400.
  warnings.warn(msg, RuntimeWarning)


Group: Alternatives ETF List (35)
Top10: ['MOM', 'SIZ', 'CHEP', 'WTMF', 'QAI']
Group: Broad Asia ETF List (44)
Top10: ['AIA', 'EEMA', 'DBAP', 'AAXJ', 'FPA']
Group: Commodity ETF List (125)
Top10: ['KOLD', 'UCO', 'BOIL', 'DTO', 'SCO']
Group: Consumer Discretionary Equity ETF List (31)
Top10: ['RETL', 'RTH', 'IYC', 'FDIS', 'VCR']
Group: Consumer Staples Equity ETF List (26)
Top10: ['UGE', 'SZK', 'PSL', 'RHS', 'VDC']
Group: Crude Oil ETF List (22)
Top10: ['UCO', 'DTO', 'SCO', 'SZO', 'OLEM']
Group: Currency ETF List (36)
Top10: ['USDU', 'FXE', 'DBV', 'FXA', 'FXS']
Group: Developed Asia Pacific ETF List (108)
Top10: ['IDOG', 'DOO', 'EWT', 'DFJ', 'SCJ']
Group: Developed Markets ETF List 1 (100)
Top10: ['IDOG', 'GII', 'WDIV', 'ACWV', 'IDLV']
Group: Developed Markets ETF List 2 (100)
Top10: ['EFZ', 'DOO', 'DUST', 'RING', 'CIL']
Group: Developed Markets ETF List 3 (93)
Top10: ['CROP', 'IPO', 'PEX', 'FTAG', 'FDTS']
Group: Emerging Asia Pacific ETF List (114)
Top10: ['YINN', 'INDL', 'FXP', 'INCO'

In [15]:
allgroup_top5_return = pd.concat([etf_date,allgroup_top5_return],axis=1)

In [19]:
allgroup_top5_return.head()

,Date,Alternatives ETF List (35),Broad Asia ETF List (44),Commodity ETF List (125),Consumer Discretionary Equity ETF List (31),Consumer Staples Equity ETF List (26),Crude Oil ETF List (22),Currency ETF List (36),Developed Asia Pacific ETF List (108),Developed Markets ETF List 1 (100),...,Municipal Bond ETF List (29),Preferred Stock ETF List (12),Real Estate ETF List (48),Target Maturity Date Corporate Bond ETF List (24),Technology Equity ETF List (75),Telecom Equity ETF List (10),Total Bond Market ETF List (82),Treasuries ETF List (51),Utilities Equity ETF List (24),Volatility ETF List (18)
0,2014-01-01,-0.001181,-0.062599,-0.009295,-0.084227,-0.020895,-0.001716,-0.012675,-0.028143,-0.028386,...,0.028300,0.046377,-0.002077,0.029821,-0.012968,-0.022846,0.026794,-0.011839,0.035191,0.104873
1,2014-02-01,0.002788,0.045846,-0.042562,0.085938,0.033221,-0.028388,0.012926,0.037423,0.054698,...,0.006666,0.020604,0.014340,0.009802,0.057473,0.000513,0.003333,-0.001868,0.040468,-0.094300
2,2014-03-01,0.002020,0.008330,-0.006595,-0.034762,0.010780,0.007164,0.012210,0.005491,0.014741,...,0.002483,0.016984,-0.007898,0.003390,0.008435,0.023623,0.003039,0.002147,0.034733,-0.026063
3,2014-04-01,0.002650,0.014225,0.003039,-0.024895,-0.003359,0.001472,-0.002046,0.011604,0.030559,...,0.011820,0.019594,0.013188,0.014958,-0.018974,-0.021313,0.011256,-0.000911,0.052254,-0.042291
4,2014-05-01,-0.000359,0.035427,-0.018863,0.016115,0.016677,-0.014523,-0.008007,0.026132,0.015232,...,0.010381,0.012940,0.016519,0.018484,0.039479,0.053959,0.009493,-0.007963,-0.006696,-0.110062


# 將allgroup_top5 先存取成CSV

In [20]:
allgroup_top5_return.to_csv("allgroup_top5_return_new.csv")

# 再次計算Riskiness R，並依照R值排序，傳回前五名的Group

In [24]:
def get_top5_Group(data_file):

    df1 = pd.read_csv(data_file)
    etf_list1 = list(df1.columns)[2:] #<----------------------------------------------加上日期
    df1 = df1.iloc[:,2:]

    etf_dic1 = {}
    for etf_name in etf_list1:
        # 過濾平均值正負
        #if df[etf_name].mean(axis=0) < 0:
        #    continue
        
        
        #-------------------------------------------------------------------------------
        
        #同上要加上時間
        
        
        arr_returns1 = df1[etf_name] - rfrate
        #arr_returns = arr_returns*1000      #調整數量級
        
        
        
        #---------------------------------------------------------------------------------
        
        guess1 = 10**(-5)
        #### 列出細項
        #print('%s指標值:  %f' %(etf_name,risk2[0]) )
        etf_dic1[etf_name] = get_riskiness_r(guess1,arr_returns1)
    
    
    sorted_x1 = sorted(etf_dic1.items(), key=lambda kv: kv[1])
        #sorted_reversed_x = list(reversed(sorted_x))
    
    # 取前5名
    top5 = sorted_x1[:5]
        # top5 = sorted_reversed_x[:5]
    #print(top5)
    
    # 取前5的ETF名稱
    etf_top5 = [etf[0] for etf in top5]

    df2 = df1[etf_top5]
    
    return df2

In [11]:
def risk(x):
    guess  = 10**(-5)
    while (guess<1000):
        risk2 = fsolve(f1,guess,x)  #引用函數f1
        if risk2[0] != guess:
            break
        guess = guess*10
    return risk2[0]
    #return np.std(x)

def sumrisk(weight,all_return):
    w = weight
    a = all_return
    v = 0
    for i in range(len(w)):
        v = v + w[i]*risk(w[i]*(all_return.iloc[:,i]))
    return v

def penalty(w):
    if min(w)<0 or max(w)>1:
        return 100000
    else:
        return 0
def portretmean(weight,all_return):
    w = weight
    a = all_return
    return np.mean(np.dot(np.array(all_return),np.array(weight)))    

def bestweight(w):
    r = []
    r.append(penalty(w)+np.abs(sum(w)-1))
    for i in range(len(w)-1):
        v =  np.abs( risk(w[i]*(all_return.iloc[:,i]))/ sumrisk(w,all_return)  \
            - (np.mean(all_return.iloc[:,i])-rfrate)/(portretmean(w,all_return)-rfrate) ) \
            + np.abs( risk(w[i+1]*(all_return.iloc[:,i+1]))/ sumrisk(w,all_return)  \
            - (np.mean(all_return.iloc[:,i+1])-rfrate)/(portretmean(w,all_return)-rfrate))
        r.append(v) 
    print('權重:',w,'誤差值:',v+penalty(w)+np.abs(sum(w)-1))
    return r

In [13]:
def fourdrop(x):
    a = []
    for i in range(len(x)):
        a.append(round(sum(x[i]),3))
    return a
x = np.array(allgroup_top5_return.iloc[:,[0]])
x = fourdrop(x)
y = np.array(allgroup_top5_return.iloc[:,[1]])
y = fourdrop(y)
all_return = pd.DataFrame([x,y]).T
all_return = all_return

In [14]:
x = np.array([-0.01,0,0.0258])
y = np.array([-0.05,0,0.134])
all_return = pd.DataFrame([x,y]).T

# 依次計算32組的權重

In [33]:
a = list(range(0, 32))
all_return = allgroup_top10_return.iloc[:,a]
n = all_return.shape[1]
#weight = [0.4,0.6]
weight = [1/n]*n
r2 = fsolve(bestweight, weight)
print(r2)

C:\Users\USER\Anaconda3\lib\site-packages\scipy\optimize\minpack.py:162: RuntimeWarning: The number of calls to function has reached maxfev = 400.
  warnings.warn(msg, RuntimeWarning)
C:\Users\USER\Anaconda3\lib\site-packages\scipy\optimize\minpack.py:162: RuntimeWarning: The iteration is not making good progress, as measured by the 
  improvement from the last ten iterations.
  warnings.warn(msg, RuntimeWarning)


權重: [0.03125 0.03125 0.03125 0.03125 0.03125 0.03125 0.03125 0.03125 0.03125
 0.03125 0.03125 0.03125 0.03125 0.03125 0.03125 0.03125 0.03125 0.03125
 0.03125 0.03125 0.03125 0.03125 0.03125 0.03125 0.03125 0.03125 0.03125
 0.03125 0.03125 0.03125 0.03125 0.03125] 誤差值: 30.749382738875873
權重: [0.03125 0.03125 0.03125 0.03125 0.03125 0.03125 0.03125 0.03125 0.03125
 0.03125 0.03125 0.03125 0.03125 0.03125 0.03125 0.03125 0.03125 0.03125
 0.03125 0.03125 0.03125 0.03125 0.03125 0.03125 0.03125 0.03125 0.03125
 0.03125 0.03125 0.03125 0.03125 0.03125] 誤差值: 30.749382738875873
權重: [0.03125 0.03125 0.03125 0.03125 0.03125 0.03125 0.03125 0.03125 0.03125
 0.03125 0.03125 0.03125 0.03125 0.03125 0.03125 0.03125 0.03125 0.03125
 0.03125 0.03125 0.03125 0.03125 0.03125 0.03125 0.03125 0.03125 0.03125
 0.03125 0.03125 0.03125 0.03125 0.03125] 誤差值: 30.749382738875873
權重: [0.03125 0.03125 0.03125 0.03125 0.03125 0.03125 0.03125 0.03125 0.03125
 0.03125 0.03125 0.03125 0.03125 0.03125 0.03125 0.03125

權重: [0.03125 0.03125 0.03125 0.03125 0.03125 0.03125 0.03125 0.03125 0.03125
 0.03125 0.03125 0.03125 0.03125 0.03125 0.03125 0.03125 0.03125 0.03125
 0.03125 0.03125 0.03125 0.03125 0.03125 0.03125 0.03125 0.03125 0.03125
 0.03125 0.03125 0.03125 0.03125 0.03125] 誤差值: 30.74938272427682
權重: [0.03125 0.03125 0.03125 0.03125 0.03125 0.03125 0.03125 0.03125 0.03125
 0.03125 0.03125 0.03125 0.03125 0.03125 0.03125 0.03125 0.03125 0.03125
 0.03125 0.03125 0.03125 0.03125 0.03125 0.03125 0.03125 0.03125 0.03125
 0.03125 0.03125 0.03125 0.03125 0.03125] 誤差值: 30.74938273561566
權重: [0.03125 0.03125 0.03125 0.03125 0.03125 0.03125 0.03125 0.03125 0.03125
 0.03125 0.03125 0.03125 0.03125 0.03125 0.03125 0.03125 0.03125 0.03125
 0.03125 0.03125 0.03125 0.03125 0.03125 0.03125 0.03125 0.03125 0.03125
 0.03125 0.03125 0.03125 0.03125 0.03125] 誤差值: 30.749382738560733
權重: [0.03125 0.03125 0.03125 0.03125 0.03125 0.03125 0.03125 0.03125 0.03125
 0.03125 0.03125 0.03125 0.03125 0.03125 0.03125 0.03125 0

權重: [0.03125 0.03125 0.03125 0.03125 0.03125 0.03125 0.03125 0.03125 0.03125
 0.03125 0.03125 0.03125 0.03125 0.03125 0.03125 0.03125 0.03125 0.03125
 0.03125 0.03125 0.03125 0.03125 0.03125 0.03125 0.03125 0.03125 0.03125
 0.03125 0.03125 0.03125 0.03125 0.03125] 誤差值: 30.749382737570457
權重: [0.03125 0.03125 0.03125 0.03125 0.03125 0.03125 0.03125 0.03125 0.03125
 0.03125 0.03125 0.03125 0.03125 0.03125 0.03125 0.03125 0.03125 0.03125
 0.03125 0.03125 0.03125 0.03125 0.03125 0.03125 0.03125 0.03125 0.03125
 0.03125 0.03125 0.03125 0.03125 0.03125] 誤差值: 30.74938275550922
權重: [0.03125 0.03125 0.03125 0.03125 0.03125 0.03125 0.03125 0.03125 0.03125
 0.03125 0.03125 0.03125 0.03125 0.03125 0.03125 0.03125 0.03125 0.03125
 0.03125 0.03125 0.03125 0.03125 0.03125 0.03125 0.03125 0.03125 0.03125
 0.03125 0.03125 0.03125 0.03125 0.03125] 誤差值: 30.749382738476218
權重: [0.03125 0.03125 0.03125 0.03125 0.03125 0.03125 0.03125 0.03125 0.03125
 0.03125 0.03125 0.03125 0.03125 0.03125 0.03125 0.03125 

# 計算出group中 前五名的權重

In [25]:
df2 = get_top5_Group("C:\\Users\\USER\\github\\HW1--data-mining\\FinalProject\\Scripts\\allgroup_top5_return_new.csv")

C:\Users\USER\Anaconda3\lib\site-packages\scipy\optimize\minpack.py:162: RuntimeWarning: The iteration is not making good progress, as measured by the 
  improvement from the last ten iterations.
  warnings.warn(msg, RuntimeWarning)
C:\Users\USER\Anaconda3\lib\site-packages\scipy\optimize\minpack.py:162: RuntimeWarning: The number of calls to function has reached maxfev = 400.
  warnings.warn(msg, RuntimeWarning)


In [28]:
df2.head()

,Developed Markets ETF List 1 (100),Preferred Stock ETF List (12),Currency ETF List (36),Municipal Bond ETF List (29),Junk ETF List (43)
0,-0.028386,0.046377,-0.012675,0.028300,-0.003431
1,0.054698,0.020604,0.012926,0.006666,0.002806
2,0.014741,0.016984,0.012210,0.002483,0.002350
3,0.030559,0.019594,-0.002046,0.011820,0.000428
4,0.015232,0.012940,-0.008007,0.010381,-0.001377


In [29]:
a = list(range(0, 5))
all_return = df2.iloc[:,a]
n = all_return.shape[1]
#weight = [0.4,0.6]
weight = [1/n]*n
r2 = fsolve(bestweight, weight)
print(r2)

權重: [0.2 0.2 0.2 0.2 0.2] 誤差值: 22.579615532835497
權重: [0.2 0.2 0.2 0.2 0.2] 誤差值: 22.579615532835497
權重: [0.2 0.2 0.2 0.2 0.2] 誤差值: 22.579615532835497
權重: [0.2 0.2 0.2 0.2 0.2] 誤差值: 22.579613133336625
權重: [0.2 0.2 0.2 0.2 0.2] 誤差值: 22.579611980380133
權重: [0.2 0.2 0.2 0.2 0.2] 誤差值: 22.579616802557073
權重: [0.2 0.2 0.2 0.2 0.2] 誤差值: 22.57961376137714
權重: [0.2 0.2 0.2 0.2 0.2] 誤差值: 22.57961528081565
權重: [ 0.52722274  0.12068872  0.40000001  0.15439011 -0.20230158] 誤差值: 100011.28979799688
權重: [0.18524552 0.2074429  0.39999959 0.37898867 0.24713835] 誤差值: 11.70862785412833
權重: [0.17048817 0.21488659 0.59999946 0.55797927 0.29428092] 誤差值: 8.364175405913093
權重: [0.14096799 0.22977527 0.99999581 0.91596125 0.38857263] 誤差值: 8.396712152030865
權重: [0.17559885 0.24480213 0.66666503 0.51557722 0.54321116] 誤差值: 7.919752315642815
權重: [0.1767956  0.26703819 0.74799174 0.50703103 0.79268142] 誤差值: 8.592338021337424
權重: [0.14096799 0.22977527 0.99999581 0.91596125 0.38857263] 誤差值: 8.396712081615195
權重: [0.1

In [ ]:
for subdir in os.listdir(data_path):
    subdir_path = os.path.join(data_path, subdir)
    if os.path.exists(subdir_path) and os.path.isdir(subdir_path):
        file = subdir + '_return.csv'
        file_path = os.path.join(subdir_path, file)